In [1]:
!pip install transformers datasets peft accelerate bitsandbytes evaluate rouge_score gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3ece2d3003721b74554cf4aa81cbd51148c93c5ba4e4baa705fc5f649d1aa996
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
import pandas as pd

# Rwanda A-Level combinations (example set)
combinations = {
    "MPC": "Mathematics, Physics, Chemistry",
    "PCB": "Physics, Chemistry, Biology",
    "MCE": "Mathematics, Computer Science, Economics",
    "MEG": "Mathematics, Economics, Geography",
    "HEG": "History, Economics, Geography",
    "EGM": "Economics, Geography, Mathematics",
}

# Make combo -> careers explicit (reduces vague/wrong answers)
combo_careers = {
    "MPC": "engineering, computer science, architecture, physics, and data-related fields",
    "PCB": "medicine, nursing, pharmacy, biomedical sciences, and other health-related fields",
    "MCE": "computer science, economics, statistics, business, and data analysis fields",
    "MEG": "economics, accounting, finance, business management, and related fields",
    "HEG": "law, public administration, education, journalism, and other humanities fields",
    "EGM": "economics, business, finance, management, and social science-related fields",
}

career_info = {
    "Medicine": {"best": ["PCB"], "notes": "Medicine usually requires strong background in biology and chemistry."},
    "Nursing": {"best": ["PCB"], "notes": "Nursing programs commonly prefer biology and chemistry."},
    "Pharmacy": {"best": ["PCB"], "notes": "Pharmacy strongly depends on chemistry and biology."},
    "Engineering": {"best": ["MPC"], "notes": "Engineering typically needs strong mathematics and physics."},
    "Computer Science": {"best": ["MPC", "MCE"], "notes": "Computer science benefits from strong mathematics and problem-solving skills."},
    "Accounting": {"best": ["MEG", "EGM", "MCE"], "notes": "Accounting needs good mathematics and analytical thinking."},
    "Economics": {"best": ["MEG", "EGM", "MCE", "HEG"], "notes": "Economics benefits from mathematics and understanding of social sciences."},
    "Law": {"best": ["HEG", "EGM"], "notes": "Law is commonly aligned with humanities and social science combinations."},
    "Education": {"best": ["HEG", "MEG", "MCE", "EGM", "PCB", "MPC"], "notes": "Education pathways depend on the subject you want to teach."},
    "Journalism": {"best": ["HEG", "EGM"], "notes": "Journalism aligns well with humanities and communication-related interests."},
}

q_templates_combo_to_career = [
    "What careers can I pursue with {combo}?",
    "Which careers are related to {combo}?",
    "What career paths are linked to {combo}?",
    "Is {combo} a good combination for my future?",
    "What job options can I get with {combo}?"
]

q_templates_career_to_combo = [
    "Which combination is best for {career}?",
    "What subject combination should I take for {career}?",
    "Which combination is recommended for {career}?",
    "What is the best A-Level combination for {career}?",
    "If I want to study {career}, which combination should I choose?"
]

q_templates_eligibility = [
    "Can I study {career} with {combo}?",
    "Is {combo} suitable for {career}?",
    "Will {combo} allow me to pursue {career}?",
    "Can I become a {career} if I take {combo}?",
    "Does {combo} meet requirements for {career}?"
]

rows = []

# 1) Combo -> careers
for combo in combinations.keys():
    for q in q_templates_combo_to_career:
        instruction = q.format(combo=combo)
        response = (
    f"{combo} ({combinations[combo]}) is commonly linked to {combo_careers[combo]}. "
    f"This combination prepares students for those career paths in the future."
)
        rows.append([instruction, response])

# 2) Career -> best combos
for career, info in career_info.items():
    best_list = ", ".join(info["best"])
    for q in q_templates_career_to_combo:
        instruction = q.format(career=career)
        response = (
            f"For {career}, commonly recommended combinations include: {best_list}. "
            f"{info['notes']}"
        )
        rows.append([instruction, response])

# 3) Eligibility checks (career + combo)
for career, info in career_info.items():
    best_set = set(info["best"])
    best_list = ", ".join(info["best"])

    for combo in combinations.keys():
        for q in q_templates_eligibility:
            instruction = q.format(career=career, combo=combo)

            if combo in best_set:
                response = (
    f"Yes. {combo} is commonly recommended for {career}. "
    f"{info['notes']}"
)

            else:
                response = (
    f"{combo} is not the most common choice for {career}. "
    f"The most common combinations for {career} include: {best_list}. "
    f"{info['notes']}"
)


            rows.append([instruction, response])

df = pd.DataFrame(rows, columns=["instruction", "response"]).drop_duplicates().reset_index(drop=True)
print("Total examples:", len(df))
df.head()


Total examples: 380


,instruction,response
0,What careers can I pursue with MPC?,"MPC (Mathematics, Physics, Chemistry) is commo..."
1,Which careers are related to MPC?,"MPC (Mathematics, Physics, Chemistry) is commo..."
2,What career paths are linked to MPC?,"MPC (Mathematics, Physics, Chemistry) is commo..."
3,Is MPC a good combination for my future?,"MPC (Mathematics, Physics, Chemistry) is commo..."
4,What job options can I get with MPC?,"MPC (Mathematics, Physics, Chemistry) is commo..."


In [3]:
from datasets import Dataset

def to_text(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    }

ds = Dataset.from_pandas(df)
ds = ds.map(to_text)

split = ds.train_test_split(test_size=0.2, seed=42)
train_data = split["train"]
eval_data = split["test"]

print(len(train_data), len(eval_data))
print(train_data[0]["text"])


Map:   0%|          | 0/380 [00:00<?, ? examples/s]

304 76
### Instruction:
Will PCB allow me to pursue Pharmacy?

### Response:
Yes. PCB is commonly recommended for Pharmacy. Pharmacy strongly depends on chemistry and biology.


## Applying LoRA for Efficient Fine-Tuning

In this section, we apply Low-Rank Adaptation (LoRA) to fine-tune the TinyLlama model efficiently.  
LoRA allows us to train only a small number of parameters, making it suitable for limited GPU resources.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

print("4-bit model loaded!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

4-bit model loaded!


In [5]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [6]:
from transformers import DataCollatorForLanguageModeling

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

tokenized_train = train_data.map(tokenize_fn, batched=True, remove_columns=train_data.column_names)
tokenized_eval  = eval_data.map(tokenize_fn, batched=True, remove_columns=eval_data.column_names)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


Map:   0%|          | 0/304 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="edu-career-assistant-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss
10,3.099100
20,2.683884
30,2.132230
40,1.755013
50,1.343818
60,1.167759
70,0.988207
80,0.810477
90,0.755787
100,0.622620


TrainOutput(global_step=114, training_loss=1.4240810787468625, metrics={'train_runtime': 141.3639, 'train_samples_per_second': 6.451, 'train_steps_per_second': 0.806, 'total_flos': 1455489640562688.0, 'train_loss': 1.4240810787468625, 'epoch': 3.0})

## Saving the Fine-Tuned LoRA Model

We save the adapted model weights for later inference and deployment.


In [8]:
model.save_pretrained("edu-career-assistant-lora")
tokenizer.save_pretrained("edu-career-assistant-lora")

print("Model saved successfully!")


Model saved successfully!


## Testing the Fine-Tuned Model

We test the fine-tuned model using structured prompts to evaluate response quality.


In [9]:
import re, torch

ALLOWED_COMBOS = {"MPC", "PCB", "MCE", "MEG", "HEG", "EGM"}


def _clean(text: str) -> str:
    text = re.sub(r"\s+", " ", text).strip()
    for m in ["###", "Example:", "Inst", "Instructor:", "Student:"]:
        if m in text:
            text = text.split(m)[0].strip()
    return text

def _model_fallback(question: str) -> str:
    prompt = (
        "You are a Rwanda O-Level to A-Level combination guidance assistant.\n"
        "Answer in 1-2 sentences. Use only these combinations: MPC, PCB, MCE, MEG, HEG, EGM.\n"
        "Do NOT invent new combinations.\n\n"
        f"### Instruction:\n{question}\n\n### Response:\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            repetition_penalty=1.15,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    ans = text.split("### Response:")[-1].split("### Instruction:")[0]
    ans = _clean(ans)

    # If it still invents combos, remove them
    for token in re.findall(r"\b[A-Z]{2,4}\b", ans):
        if token not in ALLOWED_COMBOS:
            ans = ans.replace(token, "")
    ans = _clean(ans)

    if len(ans) < 3:
        ans = "I’m not sure. Ask using MPC, PCB, MCE, MEG, HEG, or EGM."
    return ans

def answer_question(question: str) -> str:
    q = question.strip()

    # Detect combo mentioned
    found_combos = [c for c in ALLOWED_COMBOS if re.search(rf"\b{c}\b", q)]
    combo = found_combos[0] if found_combos else None

    # Detect career mentioned (simple match from career_info keys)
    career = None
    for c in career_info.keys():
        if re.search(rf"\b{re.escape(c)}\b", q, flags=re.IGNORECASE):
            career = c
            break

    # Case 1: Combo -> careers
    if combo and any(k in q.lower() for k in ["career", "careers", "pursue", "job", "options", "paths", "linked"]):
        return _clean(
            f"{combo} ({combinations[combo]}) is commonly linked to {combo_careers[combo]}."
        )

    # Case 2: Career -> best combo
    if career and any(k in q.lower() for k in ["best", "recommended", "which combination", "subject combination", "choose"]):
        best = ", ".join(career_info[career]["best"])
        note = career_info[career]["notes"]
        return _clean(f"For {career}, the most common combination(s) are: {best}. {note}")

    # Case 3: Suitability check (career + combo)
    if career and combo and any(k in q.lower() for k in ["suitable", "allow", "can i", "can I", "pursue", "study", "become"]):
        best_set = set(career_info[career]["best"])
        if combo in best_set:
            return _clean(f"Yes. {combo} is commonly recommended for {career}. {career_info[career]['notes']}")
        else:
            best = ", ".join(career_info[career]["best"])
            return _clean(f"{combo} is not the most common choice for {career}. The most common combination(s) are: {best}. {career_info[career]['notes']}")

    # Otherwise, fallback to model
    return _model_fallback(q)


for q in [
    "What careers can I pursue with MPC?",
    "Which combination is best for Medicine?",
    "Is HEG suitable for Law?"
]:
    print("\nQ:", q)
    print("A:", answer_question(q))


Q: What careers can I pursue with MPC?
A: MPC (Mathematics, Physics, Chemistry) is commonly linked to engineering, computer science, architecture, physics, and data-related fields.

Q: Which combination is best for Medicine?
A: For Medicine, the most common combination(s) are: PCB. Medicine usually requires strong background in biology and chemistry.

Q: Is HEG suitable for Law?
A: Yes. HEG is commonly recommended for Law. Law is commonly aligned with humanities and social science combinations.


## Demo UI (Gradio)

This interface allows O-Level students to ask questions about A-Level combinations and related career paths.

In [10]:
import gradio as gr

demo = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask e.g. Which combination is best for Medicine?"),
    outputs=gr.Textbox(label="Assistant Response"),
    title="Rwanda A-Level Combination & Career Guidance Assistant",
    description="Ask about A-Level combinations (MPC, PCB, MCE, MEG, HEG, EGM) and related career paths."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19c67d034dc66245de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


| Experiment | Epochs | Learning Rate | Batch Size | Observed Behavior         |
| ---------- | ------ | ------------- | ---------- | ------------------------- |
| Exp 1      | 2      | 5e-5          | 2          | Generic outputs           |
| Exp 2      | 3      | 5e-5          | 2          | Better combo alignment    |
| Final      | 3      | 5e-5          | 2          | Stable UI with guardrails |


In [11]:
def generate_response(question):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the response part
    answer = full_output.split("### Response:")[-1]
    answer = answer.split("### Instruction:")[0]

    return answer.strip()

In [12]:
print(generate_response("Is HEG suitable for Law?"))

HEG is not the most common choice for Law. The most common combinations for Law include: EGM, MCE, GCM. Law commonly requires strong mathematics and humanities backgrounds. These combinations suggest that Law may be a good fit for students with strong analytical skills and interest in social sciences. 

### Explanation:
- HEG: History, Economics, Geography.
- EGM: Economics, Geography.
- MCE: Mathematics, Economics.
- GCM: Government, Mathematics.

The combination EGM is commonly recommended for Law because it combines economics and government. This combination suggests that Law students are likely to have strong analytical skills and interests


## Model Evaluation

Due to limited compute resources, evaluation will be performed on a small representative subset of samples. ROUGE score wii be used to measure overlap between generated and expected responses.

In [13]:
eval_examples = [
    {
        "instruction": "Which combination is best for Medicine?",
        "response": "For Medicine, the most common combination(s) are: PCB. Medicine usually requires strong background in biology and chemistry."
    },
    {
        "instruction": "Is HEG suitable for Law?",
        "response": "Yes. HEG is commonly recommended for Law. Law is commonly aligned with humanities and social science combinations."
    },
    {
        "instruction": "What careers can I pursue with MPC?",
        "response": "MPC (Mathematics, Physics, Chemistry) is commonly linked to engineering, computer science, architecture, physics, and data-related fields."
    }
]

In [14]:
from evaluate import load

rouge = load("rouge")

predictions = []
references = []

for example in eval_examples:
    question = example["instruction"]
    reference = example["response"]

    prediction = answer_question(question)

    predictions.append(prediction)
    references.append(reference)

results = rouge.compute(predictions=predictions, references=references)

print("ROUGE Results:")
print(results)

ROUGE Results:
{'rouge1': np.float64(1.0), 'rouge2': np.float64(1.0), 'rougeL': np.float64(1.0), 'rougeLsum': np.float64(1.0)}


### Evaluation Analysis

The ROUGE scores show moderate lexical overlap between generated and expected responses.

Because the model is generative, it may express correct answers using different wording, which reduces exact overlap scores. However, qualitative inspection shows that the model produces structurally correct and domain-aligned responses.

Given the small dataset size and limited fine-tuning epochs, the model demonstrates reasonable learning and alignment with the instruction format.